In [1]:
import pandas as pd
import datetime as dt
import pickle
import io
import os
import matplotlib.pyplot as plt
import numpy as np
import pytz


from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
import statsmodels.api as sm

from matplotlib.dates import DateFormatter
from matplotlib.ticker import MaxNLocator
from mpl_toolkits.axes_grid1.inset_locator import inset_axes, mark_inset

from matplotlib.dates import HourLocator, DateFormatter

import glob

import matplotlib.gridspec as gridspec
from scipy.stats import linregress

### PHOENIX analysis
last update: May 22, 2025

This is supposed to work for __Bokeh__. 

Create a script that:

    - reads the data 
    - cleans it up 
    - 10min running median
    - flags for RH 
    - categorize data based on EPA limits
    - plots PM2.5 and PM10 in bokeh

### read the csv files that Nikos's code downloads
- get dataframes for each csv

In [2]:
data_path = 'data'
df_info = pd.read_csv("PHOENIX_info.csv")


for filename in os.listdir(data_path):
    if filename.lower().endswith(".csv"):
        file_path = os.path.join(data_path, filename)
        df_name = os.path.splitext(filename)[0].replace("-", "_").lower()  # sanitize and lowercase
        globals()[df_name] = pd.read_csv(file_path)
        
for _, row in df_info.iterrows():
    phoenix_id = row['PHOENIX_ID']
    
    if pd.isna(phoenix_id):
        continue  # skip rows with missing ID
    
    df_name = phoenix_id.replace("-", "_").lower()

    if df_name in globals():
        df = globals()[df_name]

        # Check if "device name" in df matches phoenix_id
        if 'device_name' in df.columns and phoenix_id in df['device_name'].values:
            # Add all columns from row to each matching row in the df
            for col in df_info.columns:
                if col != 'PHOENIX_ID':
                    df[col] = row[col]

            globals()[df_name] = df  # reassign in case you're using the updated version later

#### Now I want to clean up the data. Basically get rid of some columns and work only with the basics

In [3]:
# --- 1. Setup and load raw data ---
columns_to_keep = [
    'timestamp_local', 'met.rh', 'met.temp', 
    'pm1', 'pm25', 'pm10', 'device_name',
    'DATETIME installed (mm/dd/yy hh:mm)', 'LAT', 'LON'
]

# Load all CSVs into globals
for filename in os.listdir(data_path):
    if filename.lower().endswith(".csv"):
        file_path = os.path.join(data_path, filename)
        df_name = os.path.splitext(filename)[0].replace("-", "_").lower()
        globals()[df_name] = pd.read_csv(file_path)

#### Start from Datetime installed and let's create 10 min running median

In [4]:
# --- 1. Setup ---
columns_to_keep = [
    'timestamp_local', 'met.rh', 'met.temp', 
    'pm1', 'pm25', 'pm10', 'device_name',
    'DATETIME installed (mm/dd/yy hh:mm)', 'LAT', 'LON'
]

pm_columns = ['pm1', 'pm25', 'pm10']

# --- 2. Load raw CSVs into globals ---
for filename in os.listdir(data_path):
    if filename.lower().endswith(".csv"):
        file_path = os.path.join(data_path, filename)
        df_name = os.path.splitext(filename)[0].replace("-", "_").lower()
        globals()[df_name] = pd.read_csv(file_path)

# --- 3. Clean, filter, and compute rolling medians ---
dfs_clean = []
dfs_rm = []
titles = []

for _, row in df_info.iterrows():
    phoenix_id = row['PHOENIX_ID']
    
    if pd.isna(phoenix_id):
        continue  # skip rows with missing ID

    df_key = phoenix_id.replace("-", "_").lower()
    
    if df_key in globals():
        df_raw = globals()[df_key]

        # Check device name match
        if 'device_name' in df_raw.columns and phoenix_id in df_raw['device_name'].values:
            # Attach metadata
            for col in df_info.columns:
                if col != 'PHOENIX_ID':
                    df_raw[col] = row[col]

            # Keep only relevant columns
            df_clean = df_raw[[col for col in columns_to_keep if col in df_raw.columns]].copy()

            # Parse datetime columns
            if 'timestamp_local' in df_clean.columns:
                df_clean['timestamp_local'] = pd.to_datetime(df_clean['timestamp_local'], errors='coerce')
            if 'DATETIME installed (mm/dd/yy hh:mm)' in df_clean.columns:
                df_clean['DATETIME installed (mm/dd/yy hh:mm)'] = pd.to_datetime(
                    df_clean['DATETIME installed (mm/dd/yy hh:mm)'], errors='coerce'
                )

            # Drop pre-installation data
            if all(c in df_clean.columns for c in ['timestamp_local', 'DATETIME installed (mm/dd/yy hh:mm)']):
                install_time = df_clean['DATETIME installed (mm/dd/yy hh:mm)'].iloc[0]
                df_clean = df_clean[df_clean['timestamp_local'] >= install_time]

            # Save cleaned DataFrame
            clean_name = f"{df_key}_clean"
            globals()[clean_name] = df_clean

            # If valid for plotting, store in list
            if 'pm10' in df_clean.columns and 'timestamp_local' in df_clean.columns:
                dfs_clean.append(df_clean)
                titles.append(phoenix_id)

            # --- Rolling median calculation ---
            df_rm = df_clean.copy()

            if not pd.api.types.is_datetime64_any_dtype(df_rm['timestamp_local']):
                df_rm['timestamp_local'] = pd.to_datetime(df_rm['timestamp_local'], errors='coerce')

            df_rm = df_rm.set_index('timestamp_local').sort_index()

            for col in pm_columns:
                if col in df_rm.columns:
                    df_rm[f"{col}_rm"] = df_rm[col].rolling("10min").median()

            df_rm.reset_index(inplace=True)

            # Save rolling median DataFrame
            rm_name = f"{df_key}_rm"
            globals()[rm_name] = df_rm
            dfs_rm.append(df_rm)


### Flag for high RH 
- Coleen noticed there were issues with high RH and huge spikes. 
- Paul: these need to be cleaned up before we can start sharing data 
- for now, until we get a better to do this let's flag for high RH (>80%)

In [5]:
dfs_flagged = []

for df in dfs_rm:
    df_flagged = df.copy()
    df_flagged['rh_flag'] = (df_flagged['met.rh'] > 80).astype(bool)
    dfs_flagged.append(df_flagged)

dfs_no_rh = [df[df['rh_flag'] == False].copy() for df in dfs_flagged]

In [6]:
for i in range(len(dfs_no_rh)):
    dfs_no_rh[i]['timestamp_local'] = pd.to_datetime(dfs_no_rh[i]['timestamp_local'])
    dfs_no_rh[i] = dfs_no_rh[i].drop_duplicates(subset='timestamp_local')

### Get rid of unwanted columns for now
- create two dfs, one for PM2.5 and one for PM10

In [7]:


df_all = pd.concat(dfs_no_rh, ignore_index=True)
df_all = df_all.set_index('timestamp_local')

df_avg = (
    df_all
    .groupby('device_name')
    .resample('10min')
    .agg({
        'pm25_rm': 'mean',
        'pm10_rm': 'mean',
        'LAT': 'first',
        'LON': 'first'
    })
    .reset_index()
)

columns_to_drop = ["pm10_rm"]
df_pm25 = df_avg.drop(columns=columns_to_drop)

columns_to_drop = ["pm25_rm"]
df_pm10 = df_avg.drop(columns=columns_to_drop)

def categorize_pm10(val):
    if val <= 54:
        return 'Good'
    elif val <= 154:
        return 'Moderate'
    elif val <= 254:
        return 'Unhealthy for Sensitive Groups'
    elif val <= 354:
        return 'Unhealthy'
    elif val <= 424:
        return 'Very Unhealthy'
    else:
        return 'Hazardous'

def categorize_pm25(val):
    if val <= 9:
        return 'Good'
    elif val <= 35.4:
        return 'Moderate'
    elif val <= 55.4:
        return 'Unhealthy for Sensitive Groups'
    elif val <= 125.4:
        return 'Unhealthy'
    elif val <= 225.4:
        return 'Very Unhealthy'
    else:
        return 'Hazardous'

df_pm10['epa_category_10'] = df_pm10['pm10_rm'].apply(categorize_pm10)
df_pm25['epa_category_25'] = df_pm25['pm25_rm'].apply(categorize_pm25)

### We're going to create data for an ajax endpoint

In [10]:
import json
from pyproj import Transformer


transformer = Transformer.from_crs("epsg:4326", "epsg:3857", always_xy=True)
df_pm10['x'], df_pm10['y'] = transformer.transform(df_pm10['LON'].values, df_pm10['LAT'].values)
df_pm25['x'], df_pm25['y'] = transformer.transform(df_pm25['LON'].values, df_pm25['LAT'].values)

df_pm10['timestamp_str'] = df_pm10['timestamp_local'].dt.strftime('%Y-%m-%dT%H-%M-%S')
df_pm25['timestamp_str'] = df_pm25['timestamp_local'].dt.strftime('%Y-%m-%dT%H-%M-%S')

df_full = pd.concat([df_pm10, df_pm25], ignore_index=True)

# Ensure the output directory exists
output_dir = "output_json"
os.makedirs(output_dir, exist_ok=True)

def nonan(x):
    return list(x.dropna())

def unique(x):
    return list(pd.unique(x.dropna()))

grouped = df_full.groupby("timestamp_str").agg({
    'x': unique,
    'y': unique,
    'epa_category_10': nonan,
    'epa_category_25': nonan
})


# Iterate over each group and save as a JSON file dropna(subset=['epa_category_10','epa_category_25'])
for timestamp, group in grouped.iterrows():
    json_data = {
        "x": group["x"],
        "y": group["y"],
        "epa_category_10": group["epa_category_10"],
        "epa_category_25": group["epa_category_25"],
    }
    # Define the filename using the timestamp
    filename = f"{timestamp}.json"
    # Write the JSON file
    with open(os.path.join(output_dir, filename), "w") as json_file:
        json.dump(json_data, json_file, indent=4)
